<a href="https://colab.research.google.com/github/KavanMehtaCS/Play-Generator-LSTM/blob/main/NLP_Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project 3: Play Generation using a LSTM Recurrent Neural Network**

*   **Problem Statement**: Automatic generation of a play character by character using a LSTM recurrent neural network.
*   A recurrent neural network is a network that “feedbacks” its input and output data to almost “memorize” the correct outputs. It functions like a usual neural network where it learns from its past errors but it takes into account the output of the previous neuron as context for its next neuron which is enabled by using an internal memory. A LSTM recurrent neural netwwork builds on this idea as it counters the problem of retrieving any data from long ago by using a long term memory. This project also uses word embeddings to predict the next character and encode them into vectors like the previous sentiment analysis project.
*   This project was trained on the Shakespearen play, Romeo and Juliet, to enable the model to learn the character sequences that came after each other. The play was broken up into different sequences of 201 characters each to create a model that would recognize the next character after a sequence of 200 characters. It repeated this process 50 times and ultimately generated a play based on an entirely new given string. The project also has the ability for reading any outside files and creating text based on that text.
*   This is a very useful concept and has a widespread from autocomplete on Google (fuzzy search) and document editors to speech generation.


In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


###Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.




In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


###Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.



In [ ]:
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



###Encoding
Since this text isn't encoded yet well need to do that ourselves. We are going to encode each unique character as a different integer.



In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Function for converting numerical values into text

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.
Therfore, we will need to pick 1 more character for every single training sample to help the model recognize the next character.

In [ ]:
seq_length = 200  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 201 and split them into input and output.

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 


EXAMPLE

INPUT
know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it b

OUTPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be


Finally we need to make training batches.

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


###Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.



However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.



In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 200, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.63194025e-03 -3.08952015e-03  2.49769026e-03 ...  2.22982001e-03
    1.69012253e-03  3.25969304e-03]
  [-5.45215467e-03 -2.20347755e-03 -7.85917393e-04 ... -3.39333806e-03
    1.61740684e-03 -3.07739852e-03]
  [-4.06617858e-03 -2.29944428e-03 -1.76013168e-03 ... -2.30753701e-03
   -4.98093199e-04 -2.66354415e-03]
  ...
  [ 4.64455085e-03 -3.24609154e-03  1.03170201e-02 ... -7.44730467e-03
    3.67813441e-03 -5.88840293e-03]
  [ 1.28858886e-03 -5.03809331e-03  1.08328508e-02 ... -2.39599403e-03
    4.02543321e-03 -4.68808739e-03]
  [ 1.75672432e-03 -3.86587693e-03  6.21458050e-03 ... -1.05027494e-03
    1.94690446e-03 -4.13409015e-03]]

 [[-1.01287512e-03  2.37863837e-03  6.21187093e-04 ... -2.86992989e-03
    2.69001373e-03 -1.85820041e-03]
  [-1.08495587e-03  2.35196858e-05  2.07983889e-03 ... -2.21139542e-03
   -2.53120484e-03 -5.62389148e-04]
  [ 1.75028422e-03  6.43867301e-04 -5.83601242e-04 ...  6.17705844e-03
   -4.69713891e-03  9.72068869e-04]
  ...
  [ 7.886

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

200
tf.Tensor(
[[-0.00163194 -0.00308952  0.00249769 ...  0.00222982  0.00169012
   0.00325969]
 [-0.00545215 -0.00220348 -0.00078592 ... -0.00339334  0.00161741
  -0.0030774 ]
 [-0.00406618 -0.00229944 -0.00176013 ... -0.00230754 -0.00049809
  -0.00266354]
 ...
 [ 0.00464455 -0.00324609  0.01031702 ... -0.0074473   0.00367813
  -0.0058884 ]
 [ 0.00128859 -0.00503809  0.01083285 ... -0.00239599  0.00402543
  -0.00468809]
 [ 0.00175672 -0.00386588  0.00621458 ... -0.00105027  0.0019469
  -0.00413409]], shape=(200, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.6319402e-03 -3.0895201e-03  2.4976903e-03 -3.0725219e-03
  3.1284110e-03  1.4380623e-03 -1.1958041e-03  7.6903049e-03
 -1.3238168e-03  2.2155647e-03 -5.7516536e-03  1.2631748e-03
  2.3814095e-03 -2.3744658e-03  6.1876960e-03  4.9306685e-03
 -1.7731625e-03 -1.5326784e-03  2.9690585e-03 -9.6661353e-04
  1.1210125e-03  3.1474114e-03  3.1549074e-03  7.8878499e-04
  4.2642276e-03 -2.4324986e-03  8.2512316e-04 -1.2175217e-03
  2.9306354e-03  2.1674205e-03 -2.6884854e-03  1.2236183e-03
  1.9463529e-03  3.9519859e-03 -3.8251474e-03  3.8225027e-03
 -4.4533047e-03  3.9876389e-04 -4.1291662e-03  4.4929604e-03
 -1.6835886e-03  3.0182994e-03 -4.3322425e-03  2.7524519e-03
 -2.9273259e-03  1.1501428e-03 -1.5722650e-05  9.6379779e-04
  1.3627572e-03  3.7642493e-04 -1.4710580e-03  5.3613116e-03
  8.4287027e-04  3.0008478e-03 -3.0021376e-03 -1.7829862e-03
 -1.4676178e-04 -2.4703721e-04 -4.0537305e-03  1.4371549e-04
  1.1234693e-03  1.4021081e-03  2.2298200e-03  1.6901225e-03
  3.259693

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"Kcd-MvYZyUFfp !b'B'e?q$ZVK&s:tIA,?mBUiR'v&MMZ\nCS,:jhYo$FAwmp!DPuMhvADIGqqzCtNTPoRsTUfQNB\n:G:SkbCeF'matvStjaE3Vdnk :A p.pAk3,fbt.hc 3\nc!utgXa ,tJBetVMHFjUdVk;-$:jpS$Hb?VETpkYHGbQEDB!Kq?BEOemdhrYGg$X-rh"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.


In [ ]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Training
Finally, we will start training the model.

**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**



In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
86/86 [==============================] - 30s 298ms/step - loss: 3.0257
Epoch 2/50
86/86 [==============================] - 28s 303ms/step - loss: 2.2452
Epoch 3/50
86/86 [==============================] - 28s 309ms/step - loss: 1.9845
Epoch 4/50
86/86 [==============================] - 29s 313ms/step - loss: 1.7990
Epoch 5/50
86/86 [==============================] - 28s 311ms/step - loss: 1.6665
Epoch 6/50
86/86 [==============================] - 29s 313ms/step - loss: 1.5673
Epoch 7/50
86/86 [==============================] - 29s 314ms/step - loss: 1.4932
Epoch 8/50
86/86 [==============================] - 29s 313ms/step - loss: 1.4372
Epoch 9/50
86/86 [==============================] - 29s 317ms/step - loss: 1.3911
Epoch 10/50
86/86 [==============================] - 29s 316ms/step - loss: 1.3523
Epoch 11/50
86/86 [==============================] - 28s 311ms/step - loss: 1.3200
Epoch 12/50
86/86 [==============================] - 29s 313ms/step - loss: 1.2887
Epoch 13/50
8

###Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.



In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
checkpoint_num = 10
#model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: I  will go by thee
I  will go by thee;
For thou injurious sweet discretion, I
For Tybalt scandler, till thou trunk'd
To go anothing the belly's member,
Though us to counteffeit strange, not in Padua,
But 'eack'd with him again.

CURTIS:
They are.

GEORCE:
Where is thy wrongfully have done unto myself?
O, make them shape; they came unto my grief;
It could denave for lives in a helpey though on't.

ESCALUS:
Of what of Harry, thou hast soundleast,
But put on me and fellow the eagles!

MENENIUS:
A net false brother.

SICINIUS:
Have you collected them home:
God wrinklen welcome Henry, be gone before to would not let it know
The nature of their living with the hour;
And she, the loving kisson Tarpeians,
Who dids you have told you in this matter and will.

KING RICHARD II:
Thou hast spoke too much already: give me leave.

HENRY BOLI


##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Classification with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_classification_rnn.
3. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
4. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.